In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import yaml

# Replica Exchange Simulations with Complexes++

In this notebook we run a temperature remc simulatoin of the ubiquitin uim1 system of the binding tutorial.
The notebook will go over setting up and running an remc simulation.

# Generate CPLX file for simulation

Here we define a topology called *system* defined as two domains which are ubiquitin and uim1 respecitvely.

In [ ]:
%%writefile binding.top
# Dimensions of PBC box in angstrom
box: [100, 100, 100]
topology:
    # Definition of the first topology named ubiquitin
    Ubiquitin:
        # Take coordinates from this file
        coordinate-file: 1Q0W.pdb
        # now we define the domains in this topology
        domains:
            ubiquitin:
                type: rigid
                selection: 'protein and name CA and segid B'
    # We also need to define the UIM1 topology
    UIM1:
        coordinate-file: 1Q0W.pdb
        domains:
            uim1:
                type: rigid
                selection: 'protein and name CA and segid A'

With *pycomplexes* we can now convert the topology into a `cplx` file that complexes can understand. We place the two domains randomly inside the simulation box to ensure that no amino acid is outside of the simulation box when the simulation starts.

In [ ]:
!pycomplexes convert binding.top binding.cplx --random

# Generate Replicas

To start the simulation the only thing left is a configuration file determining the forcefield and Monte-Carlo algorithm to use. All simulations in our example will use the same starting structure. Because the simulation will be started in individual folders we define a relative path to `binding.cplx`.

Now we generate a temperature ladder from 298K to up to 400K with 20K steps.

In [ ]:
temps = np.arange(298, 400, 20).tolist()

*complexes++* expects us to have all simulations in different folders when we use REMC. Now we setup the configs for the different temperatures in the corresponding folders.

In [ ]:
for t in temps:
    dirname = 'simulation-temp-{}'.format(t)
    os.makedirs(dirname, exist_ok=True)
    
    # complexes++ understands relative paths. Here we use 
    # this to start all replicas from the same initial 
    # coordinates. File paths are going to be relative to 
    # the place where the configuration file is stored and
    # not the folder in which complexes is started!
    config = {'structure': '../binding.cplx',
              'montecarlo': {
                  'equilibration': 0,
                  # We need to chose a random seed for each replica to get unique
                  # trial moves.
                  'seed': np.random.randint(1_000_000),
                  # For two domains the cutoff algorithm will negatively impact performance
                  'short-range-cutoff': {
                      'enable': False
                  },
                  'algorithm': 'nvt',
                  'algorithm-params': {
                      # set temperatur
                      'temperatur': t,
                      'accept-func': 'metropolis'
                  }
              },
              'output': {
                  'file': 'out.xtc',
                  'log': 'out.log',
                  'stat-file': 'out.stat',
                  'freq': 5,
                  'nstructures': 100,
                  'restart-freq': -1
              }}
    
    with open(os.path.join(dirname, 'config'), 'w') as fh:
        yaml.dump(config, fh)

# Run the simulation

Now we can run the simulations. Since we only have 2 domains running on a single thread is sufficient. 
First we  have a look at the help of `complexes++` to know that the arguments we choose for the simulation later do.

In [ ]:
!complexes++ --help

The important arguments for REMC simulations are

- multidir
- replex
- replex-accept
- replex-verbosity

*multidir* is needed to tell complexes in which directories it can find the config files for each replica.
This open accepts a [wild card](https://en.wikipedia.org/wiki/Glob_(programming)). **Note** the order in which
directories are given to complexes++ also defines neighbors directly. You are responsible to give complexes++
the directories in the correct order!

*replex-accept* defines the accept function to be used. comeplxes++ supports also Hamiltonian Replica Exchange
and the NPT ensemble. If complexes++ doesn't know the function you specified it will tell you which are available.

*replex* defines after how many sweeps an exchange is attempted.

*replex-verbosity* set to *all* to enable demux post-processing.

For more details on the REMC algorithms please refer to the manual.

Now we start the simulation. Here we do a normal temperature replica exchange simulation.

In [ ]:
%%bash
complexes++ --config=config \
     --multidir simulation-temp-* \
     --replex 5 \
     --replex-accept=remc \
     --replex-stat 5 \
     --replex-verbosity all 

# Analyze Exchanges

To get an idea, how well the exchanges between our replicas were, we can make use of `pycomplexes demux`. 

## Thermodynamically Continuous

The default option of `pycomplexes demux` is to store thermodynamically continuous information.

In [ ]:
!pycomplexes demux simulation-temp-298/config remc_demux.xvg

In the resulting `.xvg` file the first column stores the time. The other columns contain the id of the structure to find at time *t* at the configuration. The output can be loaded with numpy.

In [ ]:
demux_temps = np.loadtxt("remc_demux.xvg")

We can now visualize which structure was at a given temperature over time. The structures are identified by an
index, that denotes at which temperature this structure started the simulation. By following a temperature through structure space (configuration space), you can get a thermodynamically continuous trajectory. This output can be used with the `gmx trjcat` tool from GROMACS to generate continuous XTC files.

In [ ]:
T_idx = 0  # choose different temperature here

fig, ax = plt.subplots(figsize=plt.figaspect(1./3))
ax.step(demux_temps[:, 0], demux_temps[:, T_idx+1], "o", 
         label="T={}K".format(temps[T_idx]), where="post")
ax.set(xlabel="time [ps]", ylabel="Structure idx")
ax.legend()
fig.tight_layout()

##  Structurally Continuous

Another interesting thing is how a certain structure is being exchanged. For temperature replica exchange simulation this will allow us to see how a structure passes through temperature space.

The demux tool can provide structurally continuous information with the `--structure_cont` switch.

In [ ]:
!pycomplexes demux simulation-temp-298/config remc_demux_reps.csv --structure_cont --format=csv

Here we use an alternative output to *xvg* a *csv* (comma separated value) format that can be read using pandas.

In [ ]:
demux_reps = pd.read_csv("remc_demux_reps.csv")

In the resulting *csv* the columns for each structure are called `repl-x`, with *x* being the index.

In [ ]:
demux_reps.head()

You can visualize the walk through temperature space by plotting a selected column. Please note here that we update the yticks to represent the temperatures and not the id of the thermodynamic configuration.

In [ ]:
repl_idx = 0  # choose different replica here

fig, axs = plt.subplots(figsize=plt.figaspect(1./3))
axs.step(demux_reps["time"], demux_reps["repl-{}".format(repl_idx)],
         "o-",
         label="replica-{}".format(repl_idx), 
         where="post")
axs.set(xlabel="time [ps]", ylabel="Temperature [K]",
        yticks=range(len(temps)), yticklabels=temps)
axs.legend()
fig.tight_layout()